# Online Time Tracking and Reporting with Python API

This is the code discussed over on my blog here:  
https://www.bradleysawler.com/engineering/online-time-tracking-and-reporting-with-python-api

In [1]:
import pandas as pd
import numpy as np
import requests
from requests.exceptions import HTTPError
import json
import re

import configparser
from pathlib import Path

from getpass import getpass
from requests.auth import HTTPBasicAuth
from requests.exceptions import Timeout
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError

import datetime
import locale
import myfuncts as mf
locale.setlocale(locale.LC_ALL, '')



# Settings
mf.fn_pd_options_display()
local_tz = mf.fn_local_time_zone()

C:\Users\berno\Miniconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Functions

In [2]:
def fn_strip_out_codes_from_text(string_with_codes):
    """Strip out regular expression text (codes) and return the rest.
    Add a column by applying this function with lambda
    df['new'] = df['col_with_codes'].apply(lambda x: functionname(x))
    """
    # https://stackoverflow.com/questions/44140794/python-regex-inverse-negative-match-on-a-single-line-not-on-multiple-lines-not
    qw = string_with_codes
    rx1 = r'WQA.[a-zA-Z0-9.\-\_]+|((?:(?!WQA.[a-zA-Z0-9.\-\_]+)\S)+)'
    result1 = re.findall(rx1, qw)
    rx2 = r'RE[a-zA-Z0-9.\-\_]+|((?:(?!RE[a-zA-Z0-9.\-\_]+)\S)+)'
    result2 = re.findall(rx2, ' '.join(str(word) for word in result1) )
    result2 = ' '.join(str(word) for word in result2).strip().replace('  ', ' ')
    return result2

def fn_crosstab_timesheet(df, rounded = True, decimals=1):
    """Returns a formatted table similar to a timesheet entry"""
    if rounded == True:
        duration_column = 'duration_round'
        
    else:
        duration_column = 'duration_hours'
        decimals = 2
    
    mf.fn_floatformat(decimals)

    
    return pd.crosstab([df['WBS'], df['WQA_No'], df['RE_No']], [df['dayno'], df['day']],
                       values=df[duration_column], aggfunc='sum',margins=True, margins_name='Total')



# API Documentation

* https://clockify.me/developers-api

API Base Endpoint: `https://api.clockify.me/api/v1`  
Project: `GET /workspaces/{workspaceId}/projects`  

Time entry:  
`POST /workspaces/{workspaceId}/time-entries`  
`GET /workspaces/{workspaceId}/time-entries/{id}`  
`PUT /workspaces/{workspaceId}/time-entries/{id}`  
`DELETE /workspaces/{workspaceId}/time-entries/{id}`  

# Get and response - Projects

In [3]:
# Note /user required at the end of the base endpoint
url_base = 'https://api.clockify.me/api/v1/user'
url = 'https://api.clockify.me/api/v1'

# https://docs.python.org/3/library/configparser.html
config = configparser.ConfigParser()
config.read('config.ini') # config.ini file with [clockify] and API_KEY = MyAPIKeyWithoutQuotes
X_Api_Key = config.get('clockify', 'API_KEY')


headers = {'content-type': 'application/json', 'X-Api-Key': X_Api_Key}

response = requests.get(url_base, headers=headers)

json_response_base = response.json()

json_response_base['id']

workspaceId = json_response_base['activeWorkspace']
userId = json_response_base['id']

In [4]:
api_projects = f'/workspaces/{workspaceId}/projects'

api_url = url + api_projects

response = requests.get(api_url, headers=headers)
json_response_projects = response.json()

In [5]:
json_response_projects[0].keys()

dict_keys(['id', 'name', 'hourlyRate', 'clientId', 'workspaceId', 'billable', 'memberships', 'color', 'estimate', 'archived', 'duration', 'clientName', 'note', 'template', 'public'])

# Set Project and Dates

In [6]:
# Print out a listing of projects
project_names = [i['name'] for i in json_response_projects]
print(f'Projects: {project_names}')

# STEP 1: Input the project of interest
project = project_names[3] # First one starts at zero.
print(f'Project selected: {project}')

projectId = [i['id'] for i in json_response_projects if i['name'] == project]

# STEP 2: Enter the start date of time sheet entries to retrieve
start_year = '2020'
start_month = '01'
start_day = '01'

# STEP 3: Parameter Inputs
params_num_of_entries_to_retrieve = 1000

Projects: ['201001-00654', '201001-00659', '303052-00362', 'APLNG', 'Break', 'PD']
Project selected: APLNG


# Get and response - Time Entries

In [7]:
start_utc = start_year + '-' + start_month + '-' + start_day + 'T00:00:00Z'
params={'start': start_utc, 'page-size' : params_num_of_entries_to_retrieve}

api_projects = f'/workspaces/{workspaceId}/projects'
api_time_entries = f'/workspaces/{workspaceId}/user/{userId}/time-entries'

api_url = url + api_time_entries

response = requests.get(api_url, headers=headers, params=params)
json_response_te = response.json()

# Cleaning

In [8]:
df = pd.DataFrame.from_dict(json_response_te)

df = df[['description','billable','timeInterval']]

# Split out timeInterval dict to values as the most efficient way.
# https://stackoverflow.com/questions/38231591
df = pd.concat([df.drop('timeInterval', axis=1), pd.DataFrame(df['timeInterval'].values.tolist())], axis=1)

# Convert from object to datetime
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

df['WBS'] = df['description'].str.extract(r'(\S+)')
df['WQA_No'] = df['description'].str.extract(r'(WQA.\d{5})')
df['RE_No'] = df['description'].str.extract(r'(RE\d{5}[\w]+)')

df['Doc_No'] = df['description'].str.extract(r'(WQA.\d{5}-\w{2}-\d{5})')
df['VDN_No'] = df['description'].str.extract(r'(\d{10}-\w{4}-\d{4})')
df['Doc_No'] = df['Doc_No'].fillna(df['VDN_No'])

df['Short_Desc'] = df['description'].apply(lambda x: fn_strip_out_codes_from_text(x))

# `dt` accessor used per https://stackoverflow.com/questions/34789888
df['start'] = df['start'].dt.tz_convert(local_tz)
df['end'] = df['end'].dt.tz_convert(local_tz)

df['duration'] = df['end'] - df['start']
df['duration_hours'] = df['duration'].dt.total_seconds() / 60 / 60
df['duration_round'] = round(df['duration'].dt.total_seconds() / 60 / 60 * 2) / 2

# https://docs.python.org/3/library/datetime.html
df['day'] = df['start'].dt.strftime('%a')
df['dayno'] = df['start'].dt.strftime('%w').astype('int')
df['week'] = df['start'].dt.strftime('%W').astype('int')

df.fillna('', inplace=True)

In [9]:
df.loc[1]

description       WQA.18814 RE102025-1 WQA.18814-DD-00602_000_IFP Pipes and Fittings MTO.
billable                                                                             True
start                                                           2020-04-01 10:00:00+10:00
end                                                             2020-04-01 12:30:00+10:00
duration                                                                  0 days 02:30:00
WBS                                                                             WQA.18814
WQA_No                                                                          WQA.18814
RE_No                                                                            RE102025
Doc_No                                                                 WQA.18814-DD-00602
VDN_No                                                                                   
Short_Desc                                                        Pipes and Fittings MTO.
duration_h

# Analysis

## Week Filtered


In [10]:
# Specify the ISO week number
week_no = 11

df_ts = df.loc[(df['billable'] == True) & (df['week'] == week_no)]
print(df_ts['end'].max().strftime('Week ending: %Y-%m-%d %a'))

fn_crosstab_timesheet(df_ts).to_clipboard()

display(fn_crosstab_timesheet(df_ts))

display(fn_crosstab_timesheet(df_ts, rounded=False))

Week ending: 2020-03-20 Fri


dayno                               1   2   3   4   5 Total
day                               Mon Tue Wed Thu Fri      
WBS       WQA_No    RE_No                                  
WQA.16814 WQA.16814 RE102049_WP04 nan 2.0 9.0 4.5 5.0  20.5
WQA.18502 WQA.18502 RE102036      4.5 6.0 nan nan nan  10.5
                    RE102037      4.0 nan nan nan nan   4.0
WQA.20031 WQA.20031 RE102051      1.0 nan nan 4.5 nan   5.5
Total                             9.5 8.0 9.0 9.0 5.0  40.5

dayno                                1    2    3    4    5 Total
day                                Mon  Tue  Wed  Thu  Fri      
WBS       WQA_No    RE_No                                       
WQA.16814 WQA.16814 RE102049_WP04  nan 2.00 8.69 4.58 5.00 20.27
WQA.18502 WQA.18502 RE102036      4.54 6.00  nan  nan  nan 10.54
                    RE102037      3.78  nan  nan  nan  nan  3.78
WQA.20031 WQA.20031 RE102051      1.01  nan  nan 4.44  nan  5.45
Total                             9.33 8.00 8.69 9.02 5.00 40.04

### Week Filtered - Details

In [11]:
print(df_ts['end'].max().strftime('Week ending: %Y-%m-%d %a'))
agg_gb = {'duration_round': 'sum'}
pd.pivot_table(df_ts, index=['WBS', 'Doc_No','Short_Desc'], aggfunc=agg_gb, values=['duration_round'],
              margins=True, margins_name='TOTAL HOURS')

Week ending: 2020-03-20 Fri


duration_round
WBS         Doc_No             Short_Desc                                                                      
WQA.16814                      ISO check print reviews, meetings, ISO register                             4.00
                               Weekly meeting                                                              1.00
            WQA.16814-DD-00106 Line List T1 - Reviewing Pmin set to blank                                  0.50
            WQA.16814-DD-01610 1HC-12444-01 Eng Check completed.                                           1.00
            WQA.16814-TQ-00002                                                                             8.50
                               Discussion with GN, CD. MH call to continue as proposed.                    4.50
                               Sent to Colin and MH.                                                       1.00
WQA.18502                      Processing, emails,                                                         1.00
                               Trends 13-15                                                                2.50
            WQA.18502-DD-00101 Check Print returned from MH. Issued Rev002 to MH and DT in JPI.            1.00
            WQA.18502-DD-02603 Trends 13-15 Item 8 Sent JPI request to DT for project approval.            1.00
            WQA.18502-DD-02650 Trends 13-15 Item 1 Sent Arshan for update on MMR.                          0.00
                               Trends 13-15 Item 3 Confirmed 'CALC' is not in T2 SP3D MTO.                 0.50
            WQA.18502-TQ-00039 Valve                                                                       8.50
WQA.20031                      Meeting, Project Tracker report adding JPI doc nos                          4.50
                               Weekly meeting                                                              1.00
TOTAL HOURS                                                                                               40.50

### Week Filtered - No Description

In [12]:
df_ts[df_ts['Short_Desc'].str.len() < 5][['WBS', 'Short_Desc', 'duration_round', 'end']]

,WBS,Short_Desc,duration_round,end
86,WQA.16814,,3.50,2020-03-18 13:50:00+10:00
89,WQA.16814,,3.00,2020-03-18 09:30:00+10:00
90,WQA.16814,,0.00,2020-03-18 05:16:42+10:00
91,WQA.16814,,0.00,2020-03-18 05:01:51+10:00
92,WQA.16814,,2.00,2020-03-17 16:30:00+10:00


In [13]:
df_ts[df_ts['RE_No'].str.len() < 5][['WBS', 'RE_No', 'Short_Desc', 'duration_round', 'end']]

,WBS,RE_No,Short_Desc,duration_round,end


## Description Filter

In [14]:
search_words = ['WQA.16814', 'line', 'list']

agg_gb = {'duration_round': 'sum', 'start': 'min', 'end': 'max'}

# any_words: False equals all words (AND), True equals any of the words (OR)
pd.pivot_table(mf.fn_filter_words_in_column(df, 'description', search_words, any_words=False),
               index=['week', 'WBS', 'Doc_No','Short_Desc'], aggfunc=agg_gb, values=agg_gb.keys(),
              margins=True, margins_name='TOTAL HOURS')

duration_round  \
week        WBS       Doc_No             Short_Desc                                                          
10          WQA.16814 WQA.16814-DD-00106 Line List SPF issues                                         5.50   
                                         Line List Uploaded to SPF and comments to Process.           2.50   
                                         Line List updating SPF                                       4.00   
11          WQA.16814 WQA.16814-DD-00106 Line List T1 - Reviewing Pmin set to blank                   0.50   
TOTAL HOURS                                                                                          12.50   

                                                                                                                  end  \
week        WBS       Doc_No             Short_Desc                                                                     
10          WQA.16814 WQA.16814-DD-00106 Line List SPF issues                               2020-03-11 17:09:33+10:00   
                                         Line List Uploaded to SPF and comments to Process. 2020-03-12 10:30:00+10:00   
                                         Line List updating SPF                             2020-03-10 10:00:00+10:00   
11          WQA.16814 WQA.16814-DD-00106 Line List T1 - Reviewing Pmin set to blank         2020-03-18 14:45:00+10:00   
TOTAL HOURS                                                                                 2020-03-18 14:45:00+10:00   

                                                                                                                start  
week        WBS       Doc_No             Short_Desc                                                                    
10          WQA.16814 WQA.16814-DD-00106 Line List SPF issues                               2020-03-10 15:26:06+10:00  
                                         Line List Uploaded to SPF and comments to Process. 2020-03-12 08:05:00+10:00  
                                         Line List updating SPF                             2020-03-09 16:10:00+10:00  
11          WQA.16814 WQA.16814-DD-00106 Line List T1 - Reviewing Pmin set to blank         2020-03-18 14:21:50+10:00  
TOTAL HOURS                                                                                 2020-03-09 16:10:00+10:00

In [15]:
search_words = ['18502', '']

agg_gb = {'duration_hours': 'sum', 'duration_round': 'sum', 'start': 'min', 'end': 'max'}
pd.pivot_table(mf.fn_filter_words_in_column(df_ts, 'description', search_words, any_words=False),
               index=['WBS', 'Doc_No','Short_Desc'], aggfunc=agg_gb, values=agg_gb.keys(),
              margins=True, margins_name='TOTAL HOURS')

duration_hours  \
WBS         Doc_No             Short_Desc                                                                        
WQA.18502                      Processing, emails,                                                        1.00   
                               Trends 13-15                                                               2.28   
            WQA.18502-DD-00101 Check Print returned from MH. Issued Rev002 to MH and DT in JPI.           1.00   
            WQA.18502-DD-02603 Trends 13-15 Item 8 Sent JPI request to DT for project approval.           1.03   
            WQA.18502-DD-02650 Trends 13-15 Item 1 Sent Arshan for update on MMR.                         0.08   
                               Trends 13-15 Item 3 Confirmed 'CALC' is not in T2 SP3D MTO.                0.39   
            WQA.18502-TQ-00039 Valve                                                                      8.54   
TOTAL HOURS                                                                                              14.32   

                                                                                                duration_round  \
WBS         Doc_No             Short_Desc                                                                        
WQA.18502                      Processing, emails,                                                        1.00   
                               Trends 13-15                                                               2.50   
            WQA.18502-DD-00101 Check Print returned from MH. Issued Rev002 to MH and DT in JPI.           1.00   
            WQA.18502-DD-02603 Trends 13-15 Item 8 Sent JPI request to DT for project approval.           1.00   
            WQA.18502-DD-02650 Trends 13-15 Item 1 Sent Arshan for update on MMR.                         0.00   
                               Trends 13-15 Item 3 Confirmed 'CALC' is not in T2 SP3D MTO.                0.50   
            WQA.18502-TQ-00039 Valve                                                                      8.50   
TOTAL HOURS                                                                                              14.50   

                                                                                                                      end  \
WBS         Doc_No             Short_Desc                                                                                   
WQA.18502                      Processing, emails,                                              2020-03-16 08:30:00+10:00   
                               Trends 13-15                                                     2020-03-16 14:00:00+10:00   
            WQA.18502-DD-00101 Check Print returned from MH. Issued Rev002 to MH and DT in JPI. 2020-03-16 09:30:00+10:00   
            WQA.18502-DD-02603 Trends 13-15 Item 8 Sent JPI request to DT for project approval. 2020-03-16 12:02:00+10:00   
            WQA.18502-DD-02650 Trends 13-15 Item 1 Sent Arshan for update on MMR.               2020-03-16 12:10:00+10:00   
                               Trends 13-15 Item 3 Confirmed 'CALC' is not in T2 SP3D MTO.      2020-03-16 12:33:14+10:00   
            WQA.18502-TQ-00039 Valve                                                            2020-03-17 14:00:00+10:00   
TOTAL HOURS                                                                                     2020-03-17 14:00:00+10:00   

                                                                                                                    start  
WBS         Doc_No             Short_Desc                                                                                  
WQA.18502                      Processing, emails,                                              2020-03-16 07:30:00+10:00  
                               Trends 13-15                                                     2020-03-16 09:30:00+10:00  
            WQA.18502-DD-00101 Check Print returned from MH. Issued

# SANDBOX

In [16]:
mf.fn_pd_options_display(max_rows=50)
agg_gb = {'duration_round': 'sum'}
df.loc[df['Doc_No'].str.len() > 0].groupby(['WBS','Doc_No']).agg(agg_gb)

duration_round
WBS        Doc_No                            
RE101903-1 WQA.18822-DD-00015            0.00
           WQA.19501-DD-02604            0.50
RE101904-1 WQA.16814-DD-00010            1.50
           WQA.16814-DD-00012            4.50
           WQA.16814-DD-00013            3.00
...                                       ...
WQA.18502  WQA.18502-DD-00101            1.00
           WQA.18502-DD-02603            1.00
           WQA.18502-DD-02650            0.50
           WQA.18502-TQ-00039           11.50
WQA.18814  WQA.18814-DD-00602            2.50

[153 rows x 1 columns]